In [34]:
import cv2
import numpy as np
import math
from colorbalancer import simplest_cb
from sal_detect import saliencyDetection

In [42]:
#global variables
g_k = np.array([1,4,6,4,1])/16
g_kernel= (g_k.T).dot(g_k)

In [2]:
def split_rgb(image):
    blue, green, red = cv2.split(image)
    return red, green, blue

In [36]:
#reduce image to half size
def img_reduce(image):
    outimage = cv2.filter2D(image,cv2.CV_64F,g_kernel)
    out = outimage[::2,::2]
    return out

In [37]:
# expand image to double size
def img_expand(image):
    outimage = np.zeros((image.shape[0]*2, image.shape[1]*2), dtype=np.float64)
    outimage[::2,::2]=image[:,:]
    out = cv2.filter2D(outimage,cv2.CV_64F,g_kernel)
    return out

In [38]:
#creation of Gaussian pyramid
def gauss_pyramid(image, levels):
    output = []
    output.append(image)
    tmp = image
    for i in range(0,levels):
        tmp = img_reduce(tmp)
        output.append(tmp)
    return output

In [39]:
#building a laplacian pyramid
def lapl_pyramid(gauss_pyr):
    output = []
    k = len(gauss_pyr)
    for i in range(0,k-1):
        gauss_unit = gauss_pyr[i]
        gauss_unit_expand = img_expand(gauss_pyr[i+1])
        if gauss_unit_expand.shape[0] > gauss_unit.shape[0]:
            gauss_unit_expand = np.delete(gauss_unit_expand,(-1),axis=0)
        if gauss_unit_expand.shape[1] > gauss_unit.shape[1]:
            gauss_unit_expand = np.delete(gauss_unit_expand,(-1),axis=1)
        output.append(gauss_unit - gauss_unit_expand)
    output.append(gauss_pyr.pop())
    return output

In [40]:
# blend two laplacian pyramids based on gaussian mask
def blend(lapl_pyr_white, lapl_pyr_black, gauss_pyr_mask):
    blended_pyr = []
    l= len(gauss_pyr_mask)
    for i in range(0,l):
        p1= gauss_pyr_mask[i]*lapl_pyr_white[i]
        p2=(1 - gauss_pyr_mask[i])*lapl_pyr_black[i]
        blended_pyr.append(p1 + p2)
    return blended_pyr

In [49]:
#reconstruct the image from its pyramid
def collapse(lapl_pyr):
    output = None
    output = np.zeros((lapl_pyr[0].shape[0],lapl_pyr[0].shape[1]), dtype=np.float64)
    for i in range(len(lapl_pyr)-1,0,-1):
        lap = img_expand(lapl_pyr[i])
        lapb = lapl_pyr[i-1]
        if lap.shape[0] > lapb.shape[0]:
            lap = np.delete(lap,(-1),axis=0)
        if lap.shape[1] > lapb.shape[1]:
            lap = np.delete(lap,(-1),axis=1)
        tmp = lap + lapb
        output = tmp
    return output

In [45]:
def process_channel(channel1,channel2,w1,w2,depth):
    gauss_pyr_image1 = gauss_pyramid(channel1, depth)
    gauss_pyr_image2 = gauss_pyramid(channel2, depth)
 
    lp1 = lapl_pyramid(gauss_pyr_image1)
    lp2 = lapl_pyramid(gauss_pyr_image2)
    
    fused_channel = np.array(w1)*lp1 + np.array(w2)*lp2
    
    return collapse(fused_channel)

In [46]:
def balance_channel(channel1, channel2 , weight1, weight2, depth) :
    channel = process_channel(channel1, channel2, weight1, weight2, depth)
    channel[channel < 0] = 0
    channel[channel > 255] = 255
    channel = channel.astype(np.uint8)
    return channel

In [55]:
def main(img):
    img1= simplest_cb(img,50)
  
    lab1= cv2.cvtColor(img1, cv2.COLOR_BGR2LAB)
    lab3= lab1.copy()
    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab3)

    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)

    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    lab2 = cv2.merge((cl,a,b))

    #-----Converting image from LAB Color model to RGB model--------------------
    img2 = cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

    R1 = np.double(lab1[:,:,0])/255
    WL1= cv2.Laplacian(R1,cv2.CV_64F)
    h= np.array([1,4,6,4,1])/16
    filt= (h.T).dot(h)
    WC1= cv2.filter2D(R1,cv2.CV_64F,filt)
    for i in np.where(WC1>(math.pi/2.75)):
        WC1[i]= math.pi/2.75
    WC1= (R1-WC1)*(R1-WC1)
    WS1= saliencyDetection(img1)
    sigma= 0.25
    aver= 0.5
    WE1= np.exp(-(R1-aver)**2/(2*np.square(sigma)))

    R2 = np.double(lab2[:,:,0])/255
    WL2= cv2.Laplacian(R2,cv2.CV_64F)
    h= np.array([1,4,6,4,1])/16
    filt= (h.T).dot(h)
    WC2= cv2.filter2D(R1,cv2.CV_64F,filt)
    for i in np.where(WC2>(math.pi/2.75)):
        WC2[i]= math.pi/2.75
    WC2= (R2-WC2)*(R2-WC2)
    WS2= saliencyDetection(img1)
    sigma= 0.25
    aver= 0.5
    WE2= np.exp(-(R2-aver)**2/(2*np.square(sigma)))
    W1 = (WL1 + WC1 + WS1 + WE1)/(WL1 + WC1 + WS1 + WE1 + WL2 + WC2 + WS2 + WE2)
    W2 = (WL2 + WC2 + WS2 + WE2)/(WL1 + WC1 + WS1 + WE1 + WL2 + WC2 + WS2 + WE2)
    
    levels=5
    Weight1= gauss_pyramid(W1,5)
    Weight2= gauss_pyramid(W2,5)

    (R1,G1,B1)= split_rgb(img1)
    (R2,G2,B2)= split_rgb(img2)

    depth=5 
    R = balance_channel(R1,R2,Weight1,Weight2,depth)
    G = balance_channel(G1,G2,Weight1,Weight2,depth)
    B = balance_channel(B1,B2,Weight1,Weight2,depth)
    
    result = np.zeros(img.shape,dtype=img.dtype)
    tmp = []
    tmp.append(R)
    tmp.append(G)
    tmp.append(B)
    result = cv2.merge(tmp,result)
    return result

In [56]:
img = cv2.imread('./images/underwater3.jpg')
img = cv2.resize(img, (600,600), 2,2,cv2.INTER_AREA)
result = main(img)
cv2.imshow("Input", img)
cv2.imshow("Final", result)
cv2.waitKey()
cv2.destroyAllWindows()